# Scenario Analysis: Consitency Analysis

For the execution of the script the library Pandas is used. This is imported in this step.

In [1]:
#############################################
#
# IMPORTS
#
#############################################

import pandas as pd

The key factors must be specified in a list. 
For each factor a list with the name of all possible projections is defined. 
Based on the specification of these projections, the generally possible scenarios are calculated.

In [2]:
#############################################
#
# Definition of VARIABLES
#
#############################################

key_factor_projections = [
        ["A1A", "A1B"],
        ["A2A", "A2B", "A2C"],
        ["A3A", "A3B"],
        ["A4A", "A4B"],
        ["B1A", "B1B"],
        ["B2A", "B2B"],
        ["C1A", "C1B"],
        ["C2A", "C2B"],
        ["D1A", "D1B"],
        ["D2A", "D2B"],
        ["E1A", "E1B"],
        ["E2A", "E2B"],
]

Now all required functions are implemented. Finally, these can be easily called to perform a consistency analysis.

In [3]:
#############################################
#
# Definition of FUNCTIONS
#
#############################################

def get_possible_scenarios(key_factor_projections):
    '''
        Receives an list of lists of strings with each list representing one key factor
        and each string in the list the name of one projection of the key factor.
            key_factor_projections = [[str]]

        It returns a list of lists of strings, where each list represents one possible szenario
        and the strings the projection of a key factor in the scenario.
    '''
    if len(key_factor_projections) == 1:
        return [[projection] for projection in key_factor_projections[0]]
    else:
        possible_scenarios = []
        # Get possible scenarios without the first key factor
        end_scenarios = get_possible_scenarios(key_factor_projections[1:])

        # Iterate through all projections of the first factor
        for projection in key_factor_projections[0]:

            # Combine each projection with the possible scenarions without the first key factor
            for end_scenario in end_scenarios:
                s = [projection]
                s.extend(end_scenario)
                possible_scenarios.append(s)

        # Return all possible scenario combinations
        return possible_scenarios

def read_consitency_matrix(path):
    '''
        With this method, the consistency martix is read in.
        The matrix is defined by its file path and will be returned
        as pandas-dataframe.
    '''
    return pd.read_csv(path, sep=";")

def get_consitency(a, b, consistency_matrix):
    '''
        This function function returns the consitency of given factor
        a and b as it is defined in the consitency matrix.
    '''
    # Get the row of the consistency matrix by the second projection
    row = consistency_matrix.loc[consistency_matrix["key factor"] == b]
    try:
        # Return the column value
        return int(row[a])
    except:
        # If there is a key error return 0
        return 0

def get_consitency_for_scenario(scenario, consitency_matrix):
    '''
        This functions calculates the consitency for a scenario given as list of projections.
        E.g.:
            [A1A, A2A, A3A, A4A, B1A, B2A, C1A, C2A, D1A, D2A, E1A, E2A]

        It returns the consitency as integer.
    '''
    if len(scenario) == 0:
        return 0
    first_projection = scenario[0]
    consistency = 0
    for projection in scenario[1:]:
        consistency += get_consitency(first_projection, projection, consitency_matrix)
    return consistency + get_consitency_for_scenario(scenario[1:], consitency_matrix)

def get_consistencies(scenarios, consistency_matrix):
    '''
        This function iterates over all given scenarios and calculates the consistency
        for them. Finally it returns the scenarios with their consistency in descending
        order.
    '''
    consistencies = []
    for scenario in scenarios:
        consistencies.append((scenario, get_consitency_for_scenario(scenario, consistency_matrix)))
    return sorted(consistencies, key=lambda x:(-x[1],x[0]))

def consistencies_to_file(path, consistencies):
    '''
        This method writes all consistencies to a file. It get the path
        and the scenarios with their consistency as a tuple list.
    '''
    res_df = pd.DataFrame()

    scenarios = [i[0] for i in consistencies]
    consistency_values = [i[1] for i in consistencies]

    res_df["scenario"] = scenarios
    res_df["consistency"] = consistency_values
    res_df.to_csv(path, index=False, sep="\t")

Finally, the functions defined above are called with the data from the analysis. The scenarios including their consistency are written in descending order to the file "consistencies.csv".

In [4]:
#############################################
#
# Execution of FUNCTIONS
#
#############################################

possible_scenarios = get_possible_scenarios(key_factor_projections)
consistency_matrix = read_consitency_matrix("./consistency_matrix.csv")
consistencies = get_consistencies(possible_scenarios, consistency_matrix)
consistencies_to_file("./consistencies.csv", consistencies)